<a href="https://colab.research.google.com/github/EvenSol/NeqSim-Colab/blob/master/notebooks/AI/IoT_and_Industry4.0_with_NeqSim.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# IoT and Industry 4.0 Applications with NeqSim

Industrial Internet of Things (IoT) and Industry 4.0 strategies rely on high-quality data streams, physics-based models, and actionable analytics. NeqSim can act as the digital twin core that contextualizes field measurements, predicts system behavior, and exposes insights to operations and enterprise stakeholders.

This notebook demonstrates how to:

* contextualize smart-instrument readings with NeqSim simulations,
* simulate dynamic behavior with PI-friendly time series, and
* prepare data products that can be forwarded to IoT platforms or analytics pipelines.

## Learning objectives

By the end of this notebook you will be able to:

* Map the role of NeqSim within an Industry 4.0 architecture.
* Configure digital instruments (temperature, pressure, flow) that mirror plant transmitters.
* Run a transient simulation loop and collect IoT-ready data frames.
* Derive rolling key performance indicators (KPIs) for monitoring and anomaly detection.
* Prepare payloads that can be pushed to enterprise IoT or historian services.

## 1. Process IoT and Industry 4.0 overview

Industry 4.0 combines cyber-physical systems, cloud/edge connectivity, and advanced analytics to drive resilient, autonomous operations. In a gas-processing context, the architecture typically includes:

* **Smart field devices** – digital pressure, temperature, flow, and vibration transmitters with standardized protocols.
* **Edge/field gateways** – OPC UA, MQTT, or Modbus brokers that aggregate and contextualize signals.
* **Digital twin and simulation services** – NeqSim or other models that create soft-sensors, forecasts, and scenario insights.
* **Analytics, AI, and optimization** – streaming analytics, machine learning, and prescriptive controls.
* **Enterprise applications** – operations dashboards, maintenance management, and business systems.

The diagram below illustrates how NeqSim can sit alongside IoT gateways to deliver hybrid analytics.

In [ ]:
import matplotlib.pyplot as plt
    from matplotlib.patches import FancyBboxPatch

    fig, ax = plt.subplots(figsize=(12, 3.2))
    ax.axis('off')
    ax.set_title("Conceptual data flow from instruments to enterprise value", fontsize=13, fontweight='bold', color="#1b365d")

    boxes = [
        ("Smart field devices", (0.02, 0.5), "Sensors & transmitters
(temperature, pressure, flow)"),
        ("Edge IoT gateway", (0.24, 0.5), "Protocol conversion
OPC UA / MQTT"),
        ("NeqSim digital twin", (0.46, 0.5), "Simulation & soft sensors
Physics-based context"),
        ("Analytics & AI", (0.68, 0.5), "Anomaly detection
Predictive maintenance"),
        ("Enterprise systems", (0.86, 0.5), "MES / ERP / BI
Decision support"),
    ]

    for title, (x, y), desc in boxes:
        ax.add_patch(
            FancyBboxPatch(
                (x, y - 0.18), 0.18, 0.32,
                boxstyle="round,pad=0.03",
                fc="#f3f6fb",
                ec="#2a5599",
                linewidth=1.6,
            )
        )
        ax.text(x + 0.09, y + 0.09, title, ha='center', va='center', fontsize=11, fontweight='bold', color="#1b365d")
        ax.text(x + 0.09, y - 0.02, desc, ha='center', va='center', fontsize=9, color="#333333")

    for idx in range(len(boxes) - 1):
        x_start = boxes[idx][1][0] + 0.18
        x_end = boxes[idx + 1][1][0]
        y_pos = boxes[idx][1][1]
        ax.annotate("", xy=(x_end, y_pos), xytext=(x_start, y_pos), arrowprops=dict(arrowstyle="->", linewidth=1.6, color="#4d4d4d"))

    plt.tight_layout()
    plt.show()

## 2. Setting up a NeqSim-based digital instrument rack

The following cells assemble a simplified gas-processing segment with a heater, choke valve, and slug catcher. We attach NeqSim measurement devices to mimic plant transmitters that stream temperature, pressure, and volumetric flow.

Execute the installation cell if you are running inside Google Colab.

In [ ]:
%%capture
!pip install git+https://github.com/equinor/neqsim-python.git

In [ ]:
import json
import pandas as pd
import matplotlib.pyplot as plt

from neqsim.process import (
    clearProcess,
    stream,
    heater,
    valve,
    separator,
    runProcess,
    getProcess,
)
from neqsim.thermo import fluid
from neqsim import jneqsim

### Build the process model and virtual instruments

* Define a condensate-rich gas with methane, ethane, propane, and water.
* Route the stream through a fired heater, control valve, and slug catcher separator.
* Attach NeqSim measurement devices to emulate instrument tags.

In [ ]:
clearProcess()

# Create the reservoir fluid
feed_gas = fluid('srk')
feed_gas.addComponent('methane', 0.92)
feed_gas.addComponent('ethane', 0.04)
feed_gas.addComponent('propane', 0.02)
feed_gas.addComponent('water', 0.02)
feed_gas.setMixingRule(2)

# Configure the inlet stream
feed = stream('Feed gas', feed_gas)
feed.setFlowRate(45_000.0, 'kg/hr')
feed.setTemperature(40.0, 'C')
feed.setPressure(70.0, 'bara')
feed.setCalculateSteadyState(False)

# Add a heater prior to throttling
preheater = heater('Feed preheater', feed)
preheater.setDuty(2.0e5)  # W
preheater.setCalculateSteadyState(False)

# Model a choke valve towards the pipeline
export_valve = valve('Export choke', preheater.getOutStream(), 45.0)
export_valve.setPercentValveOpening(65.0)
export_valve.setCalculateSteadyState(False)

# Slug catcher / knock-out drum
slug_catcher = separator('Slug catcher', export_valve.getOutletStream())
slug_catcher.setSeparatorLength(6.0)
slug_catcher.setInternalDiameter(1.4)
slug_catcher.setLiquidLevel(0.35)
slug_catcher.setCalculateSteadyState(False)

process_system = getProcess()

# Attach digital instruments mirroring plant transmitters
temp_sensor = jneqsim.process.measurementdevice.TemperatureTransmitter(
    'TT-501 Heater outlet temperature', preheater.getOutStream()
)
pressure_sensor = jneqsim.process.measurementdevice.PressureTransmitter(
    'PT-502 Downstream pressure', export_valve.getOutletStream()
)
flow_sensor = jneqsim.process.measurementdevice.VolumeFlowTransmitter(
    'FT-503 Gas export flow', slug_catcher.getGasOutStream()
)

for sensor in (temp_sensor, pressure_sensor, flow_sensor):
    process_system.add(sensor)

# Solve the initial steady-state condition
runProcess()

sensor_catalog = {
    'heater_outlet_temp_C': (temp_sensor, 'C'),
    'downstream_pressure_bar': (pressure_sensor, 'bar'),
    'gas_flow_m3hr': (flow_sensor, 'm^3/hr'),
}

baseline_snapshot = {
    name: sensor.getMeasuredValue(unit)
    for name, (sensor, unit) in sensor_catalog.items()
}
baseline_snapshot['separator_liquid_level_fraction'] = slug_catcher.getLiquidLevel()

baseline_snapshot

In [ ]:
pd.DataFrame([baseline_snapshot])

## 3. Dynamic instrumentation loop

We next run a 6-minute (360 second) transient simulation with time steps of one second. The heater duty and valve opening are changed to emulate operator actions. Each step captures digital instrument values, suitable for streaming into a historian or IoT message bus.

In [ ]:
process_system.setTimeStep(1.0)  # seconds

events = {
    120: 'Increase heater duty by 20%',
    240: 'Trim valve opening by 10%',
    300: 'Return heater duty to 180 kW',
}

iot_records = []
for step in range(360):
    event_flag = None
    if step == 120:
        preheater.setDuty(2.4e5)
        event_flag = events[120]
    elif step == 240:
        export_valve.setPercentValveOpening(export_valve.getPercentValveOpening() - 10.0)
        event_flag = events[240]
    elif step == 300:
        preheater.setDuty(1.8e5)
        event_flag = events[300]

    record = {
        'time_s': process_system.getTime(),
        'heater_duty_kW': preheater.getDuty() / 1.0e3,
        'heater_outlet_temp_C': temp_sensor.getMeasuredValue('C'),
        'downstream_pressure_bar': pressure_sensor.getMeasuredValue('bar'),
        'gas_flow_m3hr': flow_sensor.getMeasuredValue('m^3/hr'),
        'separator_liquid_level_fraction': slug_catcher.getLiquidLevel(),
        'event': event_flag,
    }

    iot_records.append(record)
    process_system.runTransient()

iot_frame = pd.DataFrame(iot_records)
iot_frame.head()

In [ ]:
iot_frame.tail()

### Visualize the simulated telemetry

The plots highlight how temperature, pressure, and flow respond to the operator actions. Vertical markers align with the heater duty increase, valve trim, and heater soft landing.

In [ ]:
fig, axes = plt.subplots(3, 1, figsize=(11, 9), sharex=True)

axes[0].plot(iot_frame['time_s'], iot_frame['heater_outlet_temp_C'], color='#1b9e77', label='Outlet temperature [°C]')
axes[0].set_ylabel('Temperature [°C]')
axes[0].legend(loc='best')

axes[1].plot(iot_frame['time_s'], iot_frame['downstream_pressure_bar'], color='#d95f02', label='Downstream pressure [bar]')
axes[1].set_ylabel('Pressure [bar]')
axes[1].legend(loc='best')

axes[2].plot(iot_frame['time_s'], iot_frame['gas_flow_m3hr'], color='#7570b3', label='Gas flow [m³/h]')
axes[2].set_ylabel('Flow [m³/h]')
axes[2].set_xlabel('Time [s]')
axes[2].legend(loc='best')

for ax in axes:
    for marker, label in events.items():
        ax.axvline(marker, color='#555555', linestyle='--', linewidth=1.0)
        ax.text(marker + 2, ax.get_ylim()[1] * 0.92, label, rotation=90, fontsize=9, color='#444444', va='top')

plt.tight_layout()
plt.show()

### Rolling KPIs for anomaly detection and control

With high-frequency data available, rolling statistics help detect drifts or oscillations that may go unnoticed in raw trends. The example below calculates 30-second moving averages for outlet temperature and pressure.

In [ ]:
kpi_frame = iot_frame.set_index('time_s')
kpi_frame['temperature_sma_30s'] = kpi_frame['heater_outlet_temp_C'].rolling(window=30, min_periods=1).mean()
kpi_frame['pressure_sma_30s'] = kpi_frame['downstream_pressure_bar'].rolling(window=30, min_periods=1).mean()

fig, ax = plt.subplots(2, 1, figsize=(11, 6), sharex=True)
ax[0].plot(kpi_frame.index, kpi_frame['heater_outlet_temp_C'], color='#1b9e77', alpha=0.4, label='Temperature [°C]')
ax[0].plot(kpi_frame.index, kpi_frame['temperature_sma_30s'], color='#1b9e77', linewidth=2.0, label='30 s SMA')
ax[0].set_ylabel('Temperature [°C]')
ax[0].legend(loc='best')

ax[1].plot(kpi_frame.index, kpi_frame['downstream_pressure_bar'], color='#d95f02', alpha=0.4, label='Pressure [bar]')
ax[1].plot(kpi_frame.index, kpi_frame['pressure_sma_30s'], color='#d95f02', linewidth=2.0, label='30 s SMA')
ax[1].set_ylabel('Pressure [bar]')
ax[1].set_xlabel('Time [s]')
ax[1].legend(loc='best')

for axis in ax:
    for marker in events.keys():
        axis.axvline(marker, color='#666666', linestyle='--', linewidth=1.0)

plt.tight_layout()
plt.show()

### Packaging NeqSim telemetry for IoT platforms

The last few rows of the simulation can be serialized into JSON and posted to message brokers, historians, or analytics APIs. Replace the placeholder URL with your service endpoint when deploying.

In [ ]:
payload = iot_frame.tail(5).to_dict(orient='records')
print(json.dumps(payload, indent=2)[:600])

# Example of sending to an IoT endpoint (disabled by default)
# import requests
# response = requests.post('https://your-iot-endpoint/api/v1/telemetry', json=payload, timeout=10)
# response.raise_for_status()

## 4. Industry 4.0 use cases enabled by NeqSim

* **Soft sensors and reconciliation** – combine physical transmitters with NeqSim predictions to infer unmeasured variables (e.g., dew point, compressor polytropic efficiency).
* **Predictive maintenance** – track deviations between measured and simulated conditions to flag fouling, hydrate risk, or valve erosion.
* **Closed-loop optimization** – connect NeqSim-calculated KPIs to advanced process control (APC) or AI agents that recommend set point changes.
* **Virtual commissioning** – validate new instrumentation logic, alarm limits, and PI tags before field deployment.
* **Operations training** – replay scenarios in operator training simulators (OTS) with realistic dynamics and instrument responses.

## 5. Implementation checklist

1. Catalogue plant instruments and align each tag with a NeqSim sensor or calculated property.
2. Deploy NeqSim services near the edge gateway (Docker, FastAPI, or serverless runtimes) for low-latency feedback.
3. Stream simulator outputs alongside real measurements into your historian or IoT platform.
4. Configure alerting on key deltas (measured vs. simulated) and rolling KPIs.
5. Feed curated data sets into analytics notebooks, BI dashboards, or data-science pipelines.

## 6. Further exploration

* [Real-time process monitoring for operational safety and compliance](./Real_Time_process_monitoring_for_operational_safety_and_compliance.ipynb) – complementary workflow for condition monitoring.
* [NeqSim and Seeq integration](./NeqSim_and_Seeq.ipynb) – example of hybrid analytics using an industrial analytics platform.
* [API examples](../../API/python/example/example.ipynb) – build REST endpoints for cloud-to-edge integrations.